In [ ]:
from fastai.vision.all import *

from src.learner import get_learner_task1
from src.utils import (
    load_configuration,
    create_submission,
    save_preds,
    do_fit
)

### ! Reproducibility is endured by **get_learnertask1** - it sets the seed for the learner  

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Generate the training labels

Before starting the trainig procedure please make sure to run the **src/generate_labels.py** script. <br>
This script is responsible for splitting the labeled samples into a 80/20 train and validation split. The validation set created by **src/generate_labels.py** will be used throughout training.

In [ ]:
# !python src/generate_labels.py --in_file=data/task1/train_data/annotations.csv --out_file=data/task1/train_data/annotations_labeled.csv

# The training procedure

Initial train:
 - Train on 128x128 images first only on the last linear layers and then on the whole network
 - Repeat this training procedure (starting from the weights resulted from the previous step) with inpu size of 224x224 pixels
 - Use the model trained on 224x224 images in order to predict the classes for the unlabeled images.
 
<br>

Training iteration 1:
 - Train on 128x128 images from the entire dataset (original labeled images + pseudo-labeled images)
 - Repeat the training procedure on 224x224 images using the weights obtained in the last step.
 - Use the newly trained model in order to predict again the classes for the unlabled images.
 
<br>

Training iteration 2:
 - Train on 128x128 images from the entire dataset (original labeled images + pseudo-labeled images)
 - Repeat the training procedure on 224x224 images using the weights obtained in the last step.
 - Use the newly trained model in order create the final submission.

-----

In order to reproduce this training procedure simply run cell by cell this notebook making sure to pass the desired learner configuration, number of epochs to train at each step and the learning rate.

<br>

### Note: This notebook will create a decently high number of checkpoints - the best and the last epoch results are saved separately for each training call (named `do_fit`).
If storage is an issue simply changed the name of the saved model in order to be the same from one training call to the other in order to overwrite them. For example

Change:<br>
do_fit(learn, **'task1_resnet50_128'**, epochs=10, lr=1e-3, pct_start=0.75)<br>
do_fit(learn, **'task1_resnet50_128_unfrozen'**, epochs=3, lr=slice(1e-7, 1e-5))

<br>
<br>

Into:<br>
do_fit(learn, **'task1_resnet50_128'**, epochs=10, lr=1e-3, pct_start=0.75)<br>
do_fit(learn, **'task1_resnet50_128'**, epochs=3, lr=slice(1e-7, 1e-5))

<br><br>

# Initial train (with progressive resizing)
## 128x128 px

In [ ]:
config  = load_configuration('configs/config_task1_128.yml')
learn = get_learner_task1(config)

In [ ]:
do_fit(learn, 'task1_resnet50_128', epochs=10, lr=1e-3, pct_start=0.75)

In [ ]:
learn.load('task1_resnet50_128')

In [ ]:
learn.unfreeze()

In [ ]:
do_fit(learn, 'task1_resnet50_128_unfrozen', epochs=3, lr=slice(1e-7, 1e-5), 
       fit_type='one_cycle', save_state_dict=True)

## 224x224 px 

In [ ]:
config  = load_configuration('configs/config_task1_224.yml')
learn = get_learner_task1(config)

In [ ]:
learn.model.load_state_dict(torch.load(f"{learn.model_dir}/task1_resnet50_128_unfrozen_dict.pth"))

In [ ]:
do_fit(learn, 'task1_resnet50_224', epochs=15, lr=1e-3, pct_start=0.75)

In [ ]:
learn.unfreeze()

In [ ]:
do_fit(learn, 'task1_resnet50_224_unfrozen', epochs=3, lr=slice(1e-7, 1e-6), fit_type='one_cycle')

In [ ]:
learn.load('autosaved')

In [ ]:
learn.save('task1_resnet50_224_unfrozen')

In [ ]:
save_preds(learn, config)

<br><br>

# Train - Iteration 1

## 128x128 px 

In [ ]:
config  = load_configuration('configs/config_task1_128.yml')
learn = get_learner_task1(config, iteration=1)

In [ ]:
do_fit(learn, 'task1_resnet50_128_iter1', epochs=10, lr=1e-3, pct_start=0.75)

In [ ]:
learn.unfreeze()

In [ ]:
do_fit(learn, 'task1_resnet50_128_iter1_unfrozen', epochs=3, lr=slice(1e-7, 1e-5), 
       fit_type='one_cycle', save_state_dict=True)

## 224x224 px 

In [ ]:
config  = load_configuration('configs/config_task1_224.yml')
learn = get_learner_task1(config, iteration=1)

In [ ]:
state_dict = torch.load(f"{learn.model_dir}/task1_resnet50_128_iter1_unfrozen_dict.pth")
learn.model.load_state_dict(state_dict)

In [ ]:
do_fit(learn, 'task1_resnet50_224_iter1', epochs=15, lr=1e-3, pct_start=0.75)

In [ ]:
learn.unfreeze()

In [ ]:
do_fit(learn, 'task1_resnet50_224_iter1_unfrozen', epochs=3, lr=slice(1e-7, 1e-6), 
       fit_type='one_cycle')

In [ ]:
save_preds(learn, config, iteration=1)

<br><br>

# Train - Iteration 2

## 128x128 px 

In [ ]:
config  = load_configuration('configs/config_task1_128.yml')
learn = get_learner_task1(config, iteration=2)

In [ ]:
do_fit(learn, 'task1_resnet50_128_iter2', epochs=15, lr=1e-3, pct_start=0.75)

In [ ]:
learn.unfreeze()

In [ ]:
do_fit(learn, 'task1_resnet50_128_iter2_unfrozen', epochs=3, lr=slice(1e-7, 1e-5), 
       fit_type='one_cycle', save_state_dict=True)

## 224x224 px 

In [ ]:
config  = load_configuration('configs/config_task1_224.yml')
learn = get_learner_task1(config, iteration=2)

In [ ]:
state_dict = torch.load(f"{learn.model_dir}/task1_resnet50_128_iter2_unfrozen_dict.pth")
learn.model.load_state_dict(state_dict)

In [ ]:
do_fit(learn, 'task1_resnet50_224_iter2', epochs=15, lr=3e-4, pct_start=0.75)

In [ ]:
learn.load('task1_resnet50_224_iter2')

In [ ]:
learn.unfreeze()

In [ ]:
do_fit(learn, 'task1_resnet50_224_iter2_unfrozen', epochs=5, lr=slice(1e-6, 1e-5), 
       fit_type='one_cycle', save_state_dict=True)

<br><br>

# Creating submissions 
## Submission Iteration 2 - 128x128 px

In [ ]:
config  = load_configuration('configs/config_task1_128.yml')
learn = get_learner_task1(config, iteration=2, resnet_weights=False)

In [ ]:
learn.load('task1_resnet50_128_iter2_unfrozen')

In [ ]:
learn.export('models/test_task1_resnet50_128_iter2_unfrozen.pkl')

In [ ]:
%%time

create_submission(
    path_learn='test_task1_resnet50_128_iter2_unfrozen.pkl',
    path_test_images='data/task1/val_data',
    submission_name='task1.csv'
)

## Submission Iteration 2 - 224x224 px 

In [ ]:
config  = load_configuration('configs/config_task1_224.yml')
learn = get_learner_task1(config, iteration=2, resnet_weights=False)

In [ ]:
learn.load('task1_resnet50_224_iter2_unfrozen')

In [ ]:
learn.export('models/test_task1_resnet50_224_iter2_unfrozen.pkl')

In [ ]:
%%time

create_submission(
    path_learn='test_task1_resnet50_224_iter2_unfrozen.pkl',
    path_test_images='data/task1/val_data',
    submission_name='task1.csv'
)